N-dimensional functions:

collision calculates the place where a particle with position x1 and velocity v collide with an obstacle of radius r and centre x2. If there is not collision returns "false".

velo_col, calculates de velocity after the collision, if the collision takes place at position x1, with an obstacle with centre x2, and initial velocity v.


In [1]:
function collision(x1,x2,r,v)
    v2=dot(v,v)
    b=dot((x1-x2),v)/v2
    c=(dot((x1-x2),(x1-x2))-r^2)/v2
    if(BigFloat(b^2-c)<BigFloat("0"))
        return false
    end
    t=-b-sqrt(b^2-c)
    x=v*t+x1
    return x

end

function velo_col(x1,x2,v)
    n=(x1-x2)
    n=n/norm(n)
    vn=dot(n,v)*n
    v=v-2*vn
    v=v/norm(v)
    return v
end

velo_col (generic function with 1 method)

2D effitient algorithm:

function $frac(\alpha,\epsilon)$ return the smallest integrate numbers, $p$ and $q$, such that $|\alpha−p/q|<\epsilon/q$. It use the continued fraction develpement.

nextt(x,v,r) calculates the first of the next possibilities: The interesection of a particle at a initial position $x$ and velocity $v$, with the next line of the forme $(n,y)$, where $n$ is an integrer and $y$ is the variable or the intersection of the particle an one of the obstacles of radious $r$, and center at $int(x)$, or $int(x)+e1$, or $int(x)+e2$, or $int(x)+e1+e2$.

eff(m,b,r) calculates the center of the next obstacle of radious r with which a particle will collide in a squere arrangement, if the particle have a position $x=(0,b)$ and a velocity $(vx,vy)$ such that $0<vy/vx<1$ and $vy>0$.

Lor(x,v,r) calculates using next and eff, the center of the next obstacle of radious r with which a particle with initial position x and velocity v=[vx,vy] will collide in a squere arrangement if vx>vy>0

Finally, Lorentz(x,v,r) use rotations and reflexions to generalize Lor(x,v,r) to any velocity

In [2]:
function frac(x, epsilon)
   h1, h2 = 1, 0
   k1, k2 = 0, 1
   b = x
   while abs(k1*x - h1) > epsilon
       a = ifloor(b)
       h1, h2 = a*h1 + h2, h1
       k1, k2 = a*k1 + k2, k1
       b = 1/(b - a)
   end
   return k1, h1
end



function nextt1(x,v,r) 
    e1=[1,0]
    e2=[0,1]
    nn=int(x)
    x1=x-nn      
    t=(-x1[1])/v[1]
    test=1
    if(t<0)
        t=(1-x1[1])/v[1] 
        test=2
    end
    xx1=x1+v*t
    b=xx1[2]    
    epsilon=r/v[1]
    testt=0
    if(test==1)
        if((b>-epsilon && b<epsilon))
            return nn, testt
        elseif((1-b>-epsilon) && (1-b<epsilon))
            return nn+e2, testt
        end       
    elseif(test==2)
        if((b>-epsilon && b<epsilon))
            return nn+e1, testt
        elseif(1-b>-epsilon && 1-b<epsilon) 
            return nn+e1+e2,testt
        end
    end
    testt=1
    return nn+xx1, testt
end

function eff(m, b, r)
	kn = 0
    b1=b
    epsilon=r*sqrt(m*m+1) 
    if(b < epsilon || 1 - b < epsilon)
        if b < BigFloat("0.5")
			(q, p) = frac(m, 2.*b)
		else
			(q, p) = frac(m, 2.*(1. - b))
		end
		b = mod(m*q + b, 1)
		kn += q
    end  
	while b > epsilon && 1 - b > epsilon
		if b < BigFloat("0.5")
			(q, p) = frac(m, 2.*b)
		else
			(q, p) = frac(m, 2.*(1. - b))
		end
		b = mod(m*q + b, 1)
		kn += q
	end
	q = kn
    p = int(m*q+b1)
    return [q, p]
end

function Lor(x,v,r)   #This function obtain the coordenates of the obstacle with which 
                    #a particle at a initial position x, and intial velocity v collids 
                    #if both components of the velocity are positive, and the slope is less than 1.
    x1,test=nextt1(x,v,r)
    if(test==0)
        return x1
    end
    v1=v[1]
    v2=v[2]
    m=v2/v1
    b=x1[2]
    b=b-floor(b)
    de=[0, int(b)]
    centro=eff(m,b,r)
    x2=int(x1)-de+centro
    return x2
end

function Lorentz1(x,v,r)  
    ROT=[[0 1],
        [-1 0]]  #Rotational matrix π/2 radians
    REF=[[0 1],
        [1 0]]   #Reflect matrix, change y->x and x->y
    ROT2=ROT^2
    ROT3=ROT^3
    v1=v[1]
    v2=v[2]
    vv=copy(v)
    xx=copy(x)
    m1=v2/v1
    if(norm(int(x)-x)<r)   #if a particle begin inside an obstacle, then the first collision 
                          #is considered with the same obstacle.
        return int(x)
    end
    if(m1>=0 && v2>=0)  # if the velocity is in the quadrant I 
        if(m1<=1)
            xx=Lor(xx,vv,r)
        elseif(m1>1)  
            xx=REF*xx
            vv=REF*vv
            xx=Lor(xx,vv,r)
            xx=REF*xx
            vv=REF*vv            
        end
        return xx
    elseif(m1>=0 && v2<0) #if the velocity is in the quadrant III
        xx=ROT2*xx
        vv=ROT2*vv
        if(m1<=1)
            xx=Lor(xx,vv,r)  
        elseif(m1>1)
            xx=REF*xx
            vv=REF*vv
            xx=Lor(xx,vv,r)
            xx=REF*xx
            vv=REF*vv   
        end
        xx=ROT2*xx
        vv=ROT2*vv
        return xx
    elseif(m1<0 && v2>=0) #if the velocity is in the quadrant II
        xx=ROT*xx
        vv=ROT*vv
        if(m1<-1)
            xx=Lor(xx,vv,r)            
        elseif(m1>=-1)
            xx=REF*xx
            vv=REF*vv
            xx=Lor(xx,vv,r)
            xx=REF*xx   
            vv=REF*vv 
        end     
        xx=ROT3*xx
        vv=ROT3*vv
        return xx
    elseif(m1<0 && v2<0) #if the velocity is in the quadrant IV
        xx=ROT3*xx
        vv=ROT3*vv
        if(m1<-1)
            xx=Lor(xx,vv,r)        
        elseif(m1>=-1)
            xx=REF*xx
            vv=REF*vv
            xx=Lor(xx,vv,r)
            xx=REF*xx
            vv=REF*vv 
        end   
        xx=ROT*xx
        vv=ROT3*vv
        return xx
    end
end

Lorentz1 (generic function with 1 method)

In [3]:
function nextt(x,v,r) 
    e1=[1,0]
    e2=[0,1]
    
    nn=int(x)
    x1=x-nn      
    t=(-dot(x1,e1))/dot(v,e1)
    test=1
    if(t<0)
        t=(1-dot(x1,e1))/dot(v,e1) #o =(1,b)
        test=2
    end
    xx1=x1+v*t
    b=dot(xx1,e2)    
    epsilon=r/dot(v,e1)
    if(test==1)
        if((b>-epsilon && b<epsilon))
            return nn
        elseif((1-b>-epsilon) && (1-b<epsilon))
            return nn+e2
        end       
    elseif(test==2)
        if((b>-epsilon && b<epsilon))
            return nn+e1
        elseif(1-b>-epsilon && 1-b<epsilon) 
            return nn+e1+e2
        end
    end
    return nn+xx1
end


function Lorentzviejo(x,v,r)   # I know this is not beautifull :-P
    e1={1,0}
    e2={0,1}
    x1=dot(x,e1)
    x2=dot(x,e2)
    v1=dot(v,e1)
    v2=dot(v,e2)
    m=v2/v1
    if(norm(int(x)-x)<r)   #if a particle begin inside an obstacle, then the first collision 
                          #is considered with the same obstacle.
        return int(x)
    end
        if(m>=1 && v2>0)
            xx=x2 
            yy=x1
            vx=v2
            vy=v1
            x=[xx,yy]
            v=[vx,vy]
            x=nextt(x,v,r)
            if(typeof(x[2])==Int64)
                xx2=dot(x,e2)
                xx1=dot(x,e1)
                xx=xx2 
                yy=xx1
                x=[xx,yy]
                return x
            end
            b=dot(x,e2)
            b=b-int(b)
            de=[0, 0]
            if b<0 
                b=b+1
                de=[0, 1] 
            end
            m=vy/vx
            centro=eff(m,b,r)
            x=int(x)-de+centro
            xx2=dot(x,e2)
            xx1=dot(x,e1)
            xx=xx2
            yy=xx1
            x=[xx,yy]
            return x 
        elseif (m>=1 && v2<0)
            xx=-x2
            yy=-x1
            vx=-v2
            vy=-v1
            x=[xx,yy]
            v=[vx,vy]
            x=nextt(x,v,r)
            if(typeof(x[2])==Int64)
                xx2=dot(x,e2) 
                xx1=dot(x,e1)
                xx=-xx2
                yy=-xx1
                x=[xx,yy]
                return x
            end
            b=dot(x,e2)
            b=b-int(b)
            de=[0, 0]
            if b<0 
                b=b+1
                de=[0, 1]
            end
            m=vy/vx
            centro=eff(m,b,r)
            x=int(x)-de+centro  
            xx2=dot(x,e2)
            xx1=dot(x,e1)
            xx=-xx2
            yy=-xx1 
            x=[xx,yy]
            return x
        elseif (m>=0 && m<1 && v2>0)
        #ideal
            xx=x1
            yy=x2
            vx=v1
            vy=v2
            x=[xx,yy]
            v=[vx,vy] 
            x=nextt(x,v,r)
            if(typeof(x[2])==Int64)
                xx2=dot(x,e2)
                xx1=dot(x,e1)
                xx=xx1
                yy=xx2
                x=[xx,yy]
                return x
            end
            b=dot(x,e2) 
            b=b-int(b)
            de=[0, 0]
            if b<0 
                b=b+1
                de=[0, 1]
            end
            m=vy/vx
            centro=eff(m,b,r)
            x=int(x)-de+centro 
            xx2=dot(x,e2) 
            xx1=dot(x,e1)
            xx=xx1
            yy=xx2
            x=[xx,yy]
            return x
        elseif (m>=0 && m<1 && v2<0)
            xx=-x1
            yy=-x2
            vx=-v1
            vy=-v2
            x=[xx,yy]
            v=[vx,vy]
            x=nextt(x,v,r)
            if(typeof(x[2])==Int64)
                xx2=dot(x,e2)
                xx1=dot(x,e1)
                xx=-xx1
                yy=-xx2
                x=[xx,yy]
                return x 
            end
            b=dot(x,e2)
            b=b-int(b)
            de=[0, 0]
            if b<0 
                b=b+1
                de=[0, 1]
            end
            m=vy/vx
            centro=eff(m,b,r)
            x=int(x)-de+centro 
            xx2=dot(x,e2)
            xx1=dot(x,e1)
            xx=-xx1
            yy=-xx2
            x=[xx,yy]
            return x
        elseif (m<=-1 && v2>0)
            xx=x2
            yy=-x1
            vx=v2
            vy=-v1   
            x=[xx,yy]
            v=[vx,vy]
            x=nextt(x,v,r)
            if(typeof(x[2])==Int64)
                xx2=dot(x,e2)
                xx1=dot(x,e1)
                xx=-xx2
                yy=xx1 
                x=[xx,yy]
                return x
            end
            b=dot(x,e2)
            b=b-int(b)
            de=[0, 0]
            if b<0 
                b=b+1
                de=[0, 1]
            end #160
            m=vy/vx
            centro=eff(m,b,r)
            x=int(x)-de+centro 
            xx2=dot(x,e2)
            xx1=dot(x,e1)
            xx=-xx2
            yy=xx1
            x=[xx,yy]
            return x
        elseif (m<=-1 && v2<0) 
            xx=-x2
            yy=x1
            vx=-v2
            vy=v1   
            x=[xx,yy]
            v=[vx,vy]
            x=nextt(x,v,r)
            if(typeof(x[2])==Int64)
                xx2=dot(x,e2)
                xx1=dot(x,e1)
                xx=xx2
                yy=-xx1
                x=[xx,yy]
                return x
            end
            b=dot(x,e2)
            b=b-int(b)
            de=[0, 0]
            if b<0 
                b=b+1 
                de=[0, 1]
            end
            m=vy/vx
            centro=eff(m,b,r)
            x=int(x)-de+centro 
            xx2=dot(x,e2)
            xx1=dot(x,e1)
            xx=xx2
            yy=-xx1
            x=[xx,yy] 
            return x
        elseif (m<0 && m>-1 && v2>0)
            xx=-x1
            yy=x2
            vx=-v1
            vy=v2  
            x=[xx,yy]
            v=[vx,vy]
            x=nextt(x,v,r)
            if(typeof(x[2])==Int64) 
                xx2=dot(x,e2)
                xx1=dot(x,e1)
                xx=-xx1
                yy=xx2
                x=[xx,yy]
                return x
            end
            b=dot(x,e2)
            b=b-int(b)
            de=[0, 0] 
            if b<0 
                b=b+1
                de=[0, 1]
            end
            m=vy/vx
            centro=eff(m,b,r)
            x=int(x)-de+centro 
            xx2=dot(x,e2)
            xx1=dot(x,e1)
            xx=-xx1 
            yy=xx2
            x=[xx,yy]
            return x
        elseif (m<0 && m>-1 && v2<0)
            xx=x1
            yy=-x2
            vx=v1
            vy=-v2
            x=[xx,yy]
            v=[vx,vy]
            x=nextt(x,v,r)
            if(typeof(x[2])==Int64)
                xx2=dot(x,e2)
                xx1=dot(x,e1)
                xx=xx1
                yy=-xx2
                x=[xx,yy]
                return x
            end
            b=dot(x,e2) 
            b=b-int(b)
            de=[0, 0]
            if b<0 
                b=b+1
                de=[0, 1]
            end
            m=vy/vx
            centro=eff(m,b,r)
            x=int(x)-de+centro 
            xx2=dot(x,e2) 
            xx1=dot(x,e1)
            xx=xx1
            yy=-xx2
            x=[xx,yy]
            return x
        end
end

Lorentzviejo (generic function with 1 method)

Test for cheking the time if Lorentz1 and Lorentzviejo, which is the old (no fancy) working version of the efficient algorithm in 2D.

In [5]:

M=1000
for i=1:12
time=0
time2=0
    r=BigFloat(0.1^i)
    for j=1:M
        x=rand(2).*BigFloat[1,1]
        v=rand(2).*BigFloat[1,1]-0.5*BigFloat[1,1]
        v=v/norm(v)   
        tic()
        x11=Lorentzviejo(x,v,r)
        t=toq()
        time=time+t
        tic()
        x22=Lorentz1(x,v,r)
        t2=toq()
        time2=t2+time2
  #      if(norm(x11-x22)>0)
   #         println(x, v, " ahhhhh")
        #        println(sum(x11-x22), x11, x22)  #uncoment to test if Lorentz1 is finding the good collisions
        #       x33=collision(x,x11,r,v)        #If it works, then nothing special will happens. 
        #      x44=collision(x,x22,r,v)         
       #     println(x33, x44, " hola")
       # end
    end
    println("timeLviejo=", time/M, "  timeL1=", time2/M, " ratio=", time2/time)
end


timeLviejo=0.00032743671800000023  timeL1=0.00014872301900000027 ratio=0.4542038532159981
timeLviejo=0.000663105799  timeL1=0.0007475865900000012 ratio=1.127401677269906
timeLviejo=0.009627902746999985  timeL1=0.009797376754 ratio=1.017602380440831
timeLviejo=0.005452672127999998  timeL1=0.005591698158999997 ratio=1.0254968624073484
timeLviejo=0.006365202546000009  timeL1=0.004313622489999994 ratio=0.6776881739153361
timeLviejo=0.01911507138300001  timeL1=0.01988511473399997 ratio=1.0402846181199614
timeLviejo=0.012071740962000003  timeL1=0.011554377318000006 ratio=0.9571425823641694
timeLviejo=0.012524992570999987  timeL1=0.012129273027000004 ratio=0.9684056064898416
timeLviejo=0.013493241702999998  timeL1=0.013254693470999994 ratio=0.9823209101822458
timeLviejo=0.016687311363999993  timeL1=0.016231544745 ratio=0.972687833944105
timeLviejo=0.01736146621199999  timeL1=0.016614880448 ratio=0.9569975395578076
timeLviejo=0.024230310613999998  timeL1=0.024229500595999993 ratio=0.9999665700

Now to do a simulation, we can use center=Lorentz1(x,v,r) to obtain the coordenates of the obstacle, then use x=collision(x,center,r,v) to calculate the exact point of collision, and then v=velo_col(x,center,v) to calculate the new velocity. Then the whole simulation will be:

In [21]:
function LorentzGas1(x,v,r,steps)
    for i=1:steps
        center=Lorentz1(x,v,r) 
        x=collision(x,center,r,v)
        v=velo_col(x,center,v)
    end
    return x, v
end 

LorentzGas1 (generic function with 1 method)

In [43]:
r=BigFloat("0.00001")
x=rand(2).*BigFloat[1,1]
angle=BigFloat(rand(1)[1])*pi*2.
v=[cos(angle), sin(angle)]
v=v/norm(v)
steps=1000
x,v=LorentzGas1(x,v,r,steps)


(BigFloat[8.548390000099539303590438538995435730248267040839269767519931181842171637870294e+05,3.092440000009587859027463293877189559104561643827999743830329863403010376238541e+05],BigFloat[2.104114504891284247514848978423929874433422102050767353997175482526636512194376e-01,9.776129200778092507095484430524455687165950278545815578196522143809914391472584e-01])

Or, to calculate using a maximum time instead of steps to calculate diffusion:

In [45]:
function LorentzGas2(x,v,r,time)
    t=0
    v1=copy(v)
    while(t<time)
        x1=copy(x)
        v1=copy(v)
        center=Lorentz1(x,v,r) 
        x=collision(x,center,r,v)
        v=velo_col(x,center,v)
        t=t+norm(x-x1)
    end
    t=t-time
    if(t==0)
        v1=v
    end
    x=x-t*v1
    return x, v1
end 

LorentzGas2 (generic function with 1 method)